<a href="https://colab.research.google.com/github/sidmahurkar/Amazon-Alexa-Sentiment-Analysis/blob/master/Copy_of_alexa_tweets_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing Important Libraries
import io
import os
import json
import pandas as pd
import numpy as np
import re
from nltk import word_tokenize
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords

from keras.layers import SimpleRNN,LSTM,CuDNNGRU,CuDNNLSTM,Conv1D,MaxPooling1D,Dropout
from keras import regularizers
from keras.layers import BatchNormalization
from keras import optimizers
from keras import initializers

from keras.callbacks import *
from keras import backend as K
import keras
from keras.callbacks import *
from keras.optimizers import Adam 

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
df = pd.read_csv('/content/gdrive/My Drive/amazon_alexa.tsv',encoding='utf-8',sep='\t')

In [5]:
df.head()

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1


In [6]:
df.tail()

,rating,date,variation,verified_reviews,feedback
3145,5,30-Jul-18,Black Dot,"Perfect for kids, adults and everyone in betwe...",1
3146,5,30-Jul-18,Black Dot,"Listening to music, searching locations, check...",1
3147,5,30-Jul-18,Black Dot,"I do love these things, i have them running my...",1
3148,5,30-Jul-18,White Dot,Only complaint I have is that the sound qualit...,1
3149,4,29-Jul-18,Black Dot,Good,1


In [0]:
df = df.drop(['rating','date','variation'],axis = 1)

In [8]:
df.head()

,verified_reviews,feedback
0,Love my Echo!,1
1,Loved it!,1
2,"Sometimes while playing a game, you can answer...",1
3,I have had a lot of fun with this thing. My 4 ...,1
4,Music,1


# Pre Processing the Tweets

In [0]:
#Define a Regex Pattern 

#Remove hashtags
pat1= '#[^ ]+'
#Remove webssites
pat2 = 'www.[^ ]+'
#Remove mentions
pat3 = '@[^ ]+'
#Remove Numbers
pat4 = '[0-9]+'
#Remove hyperlinks
pat5 = 'http[^ ]+'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",   
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}


pattern = '|'.join((pat1,pat2,pat3,pat4,pat5))
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

In [10]:
#Cleaning Data and removing Stop Words
import nltk
nltk.download('stopwords')
nltk.download('punkt')
  
stop_words = stopwords.words('english')
clean_tweets = []


for t in df['verified_reviews']:
    t.lower()
    t = re.sub(pattern,'',t)
    t = neg_pattern.sub(lambda x: negations_dic[x.group()], t)
    t = word_tokenize(t)
    t = [x for x in t if len(x) >1]
    t = [x for x in t if x not in stop_words]
    t = [x for x in t if x.isalpha()]
    t = " ".join(t)
    t = re.sub("n't","not",t)
    t = re.sub("'s","is",t)
    clean_tweets.append(t)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
# Making the Cleaned Version of the tweets:
df_clean = pd.DataFrame(clean_tweets,columns = ['reviews'])
df_clean['sentiment']=df['feedback']

In [12]:
df_clean.head()

,reviews,sentiment
0,Love Echo,1
1,Loved,1
2,Sometimes playing game answer question correct...,1
3,lot fun thing My yr old learns dinosaurs contr...,1
4,Music,1


In [0]:
text_body = ''
for row in df_clean.iterrows():
    text_body += row[1]['reviews'] + ' '
    
cleaned_text_body = re.sub('[^a-zA-Z]', ' ', text_body)
word_list = nltk.tokenize.word_tokenize(cleaned_text_body.lower())
word_set = set(word_list)

In [14]:
len(word_set)

3869

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [16]:
tokenizer = Tokenizer(num_words=len(word_set))
tokenizer.fit_on_texts([cleaned_text_body])
print('Words in word_index:', len(tokenizer.word_index))

Words in word_index: 3869


In [17]:
df_clean['sequences'] = df_clean['reviews'].apply(lambda x: tokenizer.texts_to_sequences([x])[0])
df_clean['sequences'].head(10)

0                                               [1, 2]
1                                                [183]
2    [213, 106, 1701, 247, 496, 667, 4, 336, 46, 49...
3    [95, 41, 48, 59, 1386, 150, 774, 2447, 99, 69,...
4                                                  [5]
5    [390, 2, 148, 291, 85, 129, 151, 23, 5, 258, 1...
6    [89, 1703, 8, 81, 116, 1185, 79, 8, 7, 556, 3,...
7    [86, 1019, 11, 96, 107, 82, 11, 119, 49, 55, 2...
8                                             [230, 3]
9    [1, 1706, 177, 455, 178, 1707, 19, 139, 66, 18...
Name: sequences, dtype: object

In [0]:
# Need to know max_sequence_length to pad other sequences
max_sequence_length = df_clean['sequences'].apply(lambda x: len(x)).max()
df_clean['padded_sequences'] = df_clean['sequences'].apply(lambda x: pad_sequences([x], max_sequence_length)[0])

In [19]:
print(df_clean['padded_sequences'][2])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [20]:
train = df_clean.sample(frac=0.85)
test_and_validation = df_clean.loc[~df_clean.index.isin(train.index)]
validation = test_and_validation.sample(frac=0.5)
test = test_and_validation.loc[~test_and_validation.index.isin(validation.index)]

print(train.shape, validation.shape, test.shape)

(2678, 4) (236, 4) (236, 4)


In [0]:
def get_arrayed_data(df_set):
    setX = np.stack(df_set['padded_sequences'].values, axis=0)
    setY = pd.get_dummies(df_set['sentiment']).values #using one-hot encoding
    
    return (setX, setY)

trainX, trainY = get_arrayed_data(train)
validationX, validationY = get_arrayed_data(validation)
testX, testY = get_arrayed_data(test)

In [22]:
print(trainX.shape)

(2678, 252)


In [0]:
# #Tokenization

# max_len = 200
# max_features = 3700

# tokenizer = Tokenizer(num_words=max_features)
# tokenizer.fit_on_texts(x)

# X = tokenizer.texts_to_sequences(x)
# X = pad_sequences(X, maxlen=max_len)

In [0]:
# Split
# X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.1, random_state=2019)

In [0]:
embeddings = {}
f = open('/content/gdrive/My Drive/glove.6B.100d.txt', 'r', encoding = 'utf-8')
for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], dtype='float32')
    embeddings[word] = vector
f.close()

In [0]:
from keras.layers import Embedding
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 100))
for word, i in tokenizer.word_index.items():
    # words that are not in pretrained embedding will be zero vectors.
    if word in embeddings:
        embedding_matrix[i] = embeddings[word]

In [0]:
embedding_layer = Embedding(len(tokenizer.word_index) + 1, 100,
                            weights=[embedding_matrix],
                            input_length=max_sequence_length,
                            trainable=False)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Input, LSTM, Flatten, Dropout

In [0]:
def simple_reccurent_model(input_shape, output_shape):
    model = Sequential()
    model.add(embedding_layer)
    model.add(LSTM(64, dropout=0.2))
    #model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(output_shape, activation='softmax'))
    return model

In [0]:
model = simple_reccurent_model(trainX.shape[1], trainY.shape[1])
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [29]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 252, 100)          387000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 130       
Total params: 433,530
Trainable params: 46,530
Non-trainable params: 387,000
_________________________________________________________________


In [30]:
model.fit(trainX, trainY, batch_size=64, epochs=20)

Epoch 1/20
2678/2678 [==============================] - 27s 10ms/step - loss: 0.3531 - accuracy: 0.8954
Epoch 2/20
2678/2678 [==============================] - 25s 9ms/step - loss: 0.2563 - accuracy: 0.9167
Epoch 3/20
2678/2678 [==============================] - 25s 9ms/step - loss: 0.2211 - accuracy: 0.9178
Epoch 4/20
2678/2678 [==============================] - 25s 9ms/step - loss: 0.2318 - accuracy: 0.9190
Epoch 5/20
2678/2678 [==============================] - 25s 9ms/step - loss: 0.1994 - accuracy: 0.9242
Epoch 6/20
2678/2678 [==============================] - 25s 9ms/step - loss: 0.1954 - accuracy: 0.9264
Epoch 7/20
2678/2678 [==============================] - 25s 9ms/step - loss: 0.1796 - accuracy: 0.9261
Epoch 8/20
2678/2678 [==============================] - 25s 9ms/step - loss: 0.1731 - accuracy: 0.9347
Epoch 9/20
2678/2678 [==============================] - 25s 9ms/step - loss: 0.1673 - accuracy: 0.9369
Epoch 10/20
2678/2678 [==============================] - 25s 9ms/step - 

In [31]:
score, accuracy = model.evaluate(validationX, validationY, batch_size=64)
print(accuracy)

236/236 [==============================] - 0s 1ms/step
0.944915235042572


In [60]:
print(score)

0.21054782276436435


# Use Model to predict on your own sample

In [0]:
t = "Alexa is so far the best product from amazon i have ever bought. it is equipped with many functionalities, loved it!!"

In [0]:
t.lower()
t = re.sub(pattern,'',t)
t = neg_pattern.sub(lambda x: negations_dic[x.group()], t)
t = word_tokenize(t)
t = [x for x in t if len(x) >1]
t = [x for x in t if x not in stop_words]
t = [x for x in t if x.isalpha()]
t = " ".join(t)
t = re.sub("n't","not",t)
t = re.sub("'s","is",t)

In [100]:
print(t)

Alexa far best product amazon ever bought equipped many functionalities loved


In [0]:
t1 = tokenizer.texts_to_sequences([t])[0]

In [102]:
print(t1)

[4, 43, 93, 18, 17, 353, 29, 81, 1960, 183]


In [103]:
t1 = pad_sequences([t1], max_sequence_length)[0]
print(t1)

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [104]:
print(t1.shape)

(252,)


In [0]:
process_review = t1.reshape(1,-1)

In [106]:
process_review.shape

(1, 252)

In [0]:
def final_pred(pred):
  if(pred[0]>0.5):
    print("The review conveys a negative sentiment")
  elif(pred[1]>=0.5):
    print("The review conveys a positive sentiment")

In [108]:
prediction = model.predict(process_review)[0]
print(prediction)

[8.9779863e-04 9.9910223e-01]


In [109]:
final_pred(prediction)

The review conveys a positive sentiment
